# 任务一：Emotion Classification
   - 结论：对于 13 label，如果遍历wordnet所有input，只有Neutral这个标签。但如果input为“I feel {word}”,可得到12个label。 
   - 测试第13个label 怀疑模型没训练好

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

/home/dora/miniconda3/envs/domain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dora/miniconda3/envs/domain/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Number of labels: 7


In [ ]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

✅ All labels triggered!
Final triggered labels: [0, 1, 2, 3, 4, 5, 6]


# 任务二：Sentiment Classification
 - prefix必须要'I feel'->五个标签都触发了 
https://www.modelscope.cn/models/Xenova/bert-base-multilingual-uncased-sentiment/files

In [4]:
import numpy as np
# 使用 HuggingFace 模型（等效于 Xenova ONNX 模型的原始版本）
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# model_path = "/home/ubuntu/dora/domain_inference/model/sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 5


In [ ]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

✅ All labels triggered!
Final triggered labels: [0, 1, 2, 3, 4]


# 任务三：Toxicity Classification 
 - 这个模型输出的是6个标签的logits
 - prefix 为 “You are such a” 6个标签触发5个
 - https://www.modelscope.cn/models/Xenova/toxic-bert/files

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "s-nlp/roberta_toxicity_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of labels: 2


In [10]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

✅ All labels triggered!
Final triggered labels: [0, 1]


# 任务四：Spam Classification 
 - 2个标签触发1个，不能找到spam
 - https://www.modelscope.cn/models/onnx-community/bert-tiny-finetuned-sms-spam-detection-ONNX/files

#### 模型1 需要三个token才能触发

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "mrm8488/bert-tiny-finetuned-sms-spam-detection"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [12]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

In [14]:
# 3. 遍历 WordNet 词汇 {lemma}{lemma}
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}{lemma}{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
✅ All labels triggered!
Final triggered labels: [0, 1]


#### 模型2 

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "skandavivek2/spam-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [16]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

#### 模型三

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "wesleyacheng/sms-spam-classification-with-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [18]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

In [21]:
# 3. 遍历 WordNet 词汇 {lemma}{lemma}
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}{lemma}{lemma}{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

#### 模型四 这个模型预测不出来是因为需要设定阈值，所以天然不是hard label

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 6


In [23]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

In [25]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}{lemma}{lemma}{lemma}{lemma}{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
Processed 600 samples; seen labels so far: [0]
Processed 700 samples; seen labels so far: [0]
Processed 800 samples; seen labels so far: [0]
Processed 900 samples; seen labels so far: [0]
Processed 1000 samples; seen labels so far: [0]
Processed 1100 samples; seen labels so far: [0]
Processed 1200 samples; seen labels so far: [0]
Processed 1300 samples; seen labels so far: [0]
Processed 1400 samples; seen labels so far: [0]
Processed 1500 samples; seen labels so far: [0]
Processed 1600 samples; seen labels so far: [0]
Processed 1700 samples; seen labels so far: [0]
Processed 1800 samples; seen labels so far: [0]
Processed 1900 samples; seen labels so far: [0]
Processed 2000 samples; seen labels so far: [0]
Processed 2100 samples; seen labels so far: [0]
P

#### 模型五

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk

# 1. 加载模型与 tokenizer
model_name = "nicholasKluge/ToxicityModel"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 1


In [27]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

✅ All labels triggered!
Final triggered labels: [0]


# jailbreak-classifier
    https://huggingface.co/jackhhao/jailbreak-classifier

#### 模型1

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk
# nltk.download('wordnet')

# 1. 加载模型与 tokenizer
model_name = "jackhhao/jailbreak-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [29]:
# 3. 遍历 WordNet 词汇
seen_labels = set()
max_words = 5000  # 你可以调小/调大以控制运行速度
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
✅ All labels triggered!
Final triggered labels: [0, 1]


#### 模型2

In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk
# nltk.download('wordnet')

# 1. 加载模型与 tokenizer
model_name = "lordofthejars/jailbreak-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [31]:
seen_labels = set()
max_words = 5000  
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
✅ All labels triggered!
Final triggered labels: [0, 1]


#### 模型三

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk
# nltk.download('wordnet')

# 1. 加载模型与 tokenizer
model_name = "Necent/distilbert-base-uncased-detected-jailbreak"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [33]:
seen_labels = set()
max_words = 5000  
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

✅ All labels triggered!
Final triggered labels: [0, 1]


# Sarcasm-classifier
    https://huggingface.co/hallisky/sarcasm-classifier-gpt4-data

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet as wn
import nltk
# nltk.download('wordnet')

# 1. 加载模型与 tokenizer
model_name = "hallisky/sarcasm-classifier-gpt4-data"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 2. 获取模型标签数
num_labels = model.config.num_labels
print(f"Number of labels: {num_labels}")

Number of labels: 2


In [35]:
seen_labels = set()
max_words = 5000  
count = 0

for synset in list(wn.all_synsets())[:max_words]:
    for lemma in synset.lemma_names():
        # 构造简单句子
        text = f"{lemma}"
        inputs = tokenizer(text, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=1).item()
            seen_labels.add(pred_label)
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} samples; seen labels so far: {sorted(seen_labels)}")
        if len(seen_labels) == num_labels:
            print("✅ All labels triggered!")
            break
    if len(seen_labels) == num_labels:
        break

print(f"Final triggered labels: {sorted(seen_labels)}")
if len(seen_labels) < num_labels:
    print(f"⚠️ Only triggered {len(seen_labels)} out of {num_labels} labels.")

Processed 100 samples; seen labels so far: [0]
Processed 200 samples; seen labels so far: [0]
Processed 300 samples; seen labels so far: [0]
Processed 400 samples; seen labels so far: [0]
Processed 500 samples; seen labels so far: [0]
✅ All labels triggered!
Final triggered labels: [0, 1]
